In [ ]:
!pip install langchain openai langchain-openai langchain-community -q

In [ ]:
import re
import pandas as pd
from tqdm import tqdm
from getpass import getpass

from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

## Если используете ключ от OpenAI, запустите эту ячейку 👇

In [ ]:
import os
from langchain_openai import ChatOpenAI


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇


In [ ]:
from langchain_openai import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ, полученный в боте курса"
course_api_key = getpass(prompt='Введите ваш API ключ, полученный в боте курса')

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini',
                 base_url="https://aleron-llm.neuraldeep.tech/")

Введите ваш API ключ, полученный в боте курса··········


## Задание 3.2.9 🤔 Кажется, это что-то на LLM-ском? 🧐

In [ ]:
df = pd.read_csv("https://stepik.org/media/attachments/lesson/1110883/raw_texts.csv")
df.head()

,raw_text
0,"The sun was setting, casting long shadows over..."
1,"Le soleil se couchait, jetant de longues ombre..."
2,"El sol se estaba poniendo, proyectando largas ..."
3,"La ciudad estaba llena de vida, sus calles lle..."
4,"La ville était pleine de vie, ses rues remplie..."


Напишем функцию, которая очистит текст от ненужных символов: `¿, ¡, £`

In [ ]:
def clean_text(inputs: dict) -> dict:
    text = inputs["text"]

    # YOUR CODE HERE ...
    text = re.sub(r'[¿¡£]', '', text)

    return {"text": text}

In [ ]:
df['raw_text']

,raw_text
0,"The sun was setting, casting long shadows over..."
1,"Le soleil se couchait, jetant de longues ombre..."
2,"El sol se estaba poniendo, proyectando largas ..."
3,"La ciudad estaba llena de vida, sus calles lle..."
4,"La ville était pleine de vie, ses rues remplie..."
5,"Die Stadt¿ war voller Leben, ihre St£raß¿en ge..."
6,Die Sonne g¿ing unter und warf lange Schat£ten...
7,"В тихом уголке старого города, где узкие уло¿ч..."
8,In a small town nestled between the mountains ...
9,En un pequeño pueblo situado entre la£s m¿onta...


In [ ]:
print(clean_text({'text': df['raw_text'][5]}))

{'text': 'Die Stadt war voller Leben, ihre Straßen gefüllt mit dem Lärm von Menschen und Autos. Mitten im Chaos versuchte Johann, ein junger Mann mit einem Traum, seinen Weg zu machen. Seine Freunde, Peter und Maria, standen ihm immer zur Seite und unterstützten ihn bei seinen Bemühungen. Sie waren ein Trio, unzertrennlich und voller Leben. Aber das Leben in der Stadt war nicht einfach. Sie mussten viele Herausforderungen meistern, aber sie gaben nie auf. Sie glaubten an ihre Träume und aneinander. Und so setzten sie ihre Reise fort, jedes Hindernis mit Mut und Entschlossenheit zu überwinden.'}


Будем просить у модели определять язык и имя главного персонажа и выдавать ответ в виде словаря. Для этого создадим `Output parser`, с которым вы уже познакомились в прошлых уроках. НЕ ОБЯЗАТЕЛЬНЫЙ ШАГ

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
# Определим схемы ответа
language_schema = ResponseSchema(
    name="language",
    description="""язык, на котором написан текст. Указать на английском языке (например, не Русский, а Russian. Не Deutsch, а German)""")

person_schema = ResponseSchema(
    name="person",
    description="""Имя главного героя произведения. На том же языке, как оно указано в тексте. (Hans, Pierrot, Анна)""")

response_schemas = [language_schema, person_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas) # Создаём парсер и подаём в него список со схемами
format_instructions = output_parser.get_format_instructions() # Получаем инструкции по форматированию ответа

In [ ]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"language": string  // язык, на котором написан текст. Указать на английском языке (например, не Русский, а Russian. Не Deutsch, а German)
	"person": string  // Имя главного героя произведения. На том же языке, как оно указано в тексте. (Hans, Pierrot, Анна)
}
```


Напишем шаблон промпта со своим вопросом и инструкциями по форматированию ответа. Будем передавать в этот промпт сырой текст

In [40]:
template = '''Из текста извлеки информацию:
language: Определи язык текста, указать на английском языке (например, не Русский, а Russian. Не Deutsch, а German).
person: И имя главного героя произведения, На том же языке, как оно указано в тексте. (Hans, Pierrot, Анна).
text: {text}
По инструкции: {format_instructions}'''

prompt = PromptTemplate(
    template=template,
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions}
)

In [41]:
text = 'Die Stadt¿ war voller Leben, ihre St£raß¿en gefüllt mit dem Lärm von Menschen und Autos. Mitten im Chaos versuchte Joh¿ann, ein junger Man¿n mit einem Traum, seinen Weg zu machen. Seine Freunde, Peter und Maria, standen ihm immer zur Seite und unterstützten ihn bei seinen Bemühu¡ngen. Sie waren ein Trio, unzertrennlich und voller Leben. Aber das Leben in der Stadt war nicht einfach. Si¡e mussten viele Herausforderungen meistern, aber sie gaben nie auf. Sie glaubten an ihre Träume und aneinander. Und so setzten sie ihre Reise fort, jedes Hindernis mit Mut und Entschlossenheit zu überwinden.'

# Форматируем шаблон
prompt_ = template.format(text=text, format_instructions=format_instructions)

In [42]:
print(llm.invoke(prompt_).content)

```json
{
	"language": "German",
	"person": "Johann"
}
```


Создадим цепочку с помощью `LCEL`

In [43]:
chain = clean_text | prompt | llm | output_parser

print(chain.invoke({'text': df['raw_text'][5]}))

{'language': 'German', 'person': 'Johann'}


In [57]:

dict_list = [] # Список, где будем хранить ответы модели

for text in tqdm(df['raw_text']):
    cleaned_data = clean_text({'text': text})
    cleaned_text = cleaned_data['text']
    # YOUR CODE HERE
    answer = chain.invoke({'text': text}) # Добавляем сообщение в промпт
    # Создаем полный словарь с результатами
    result = {
        'cleaned_text': cleaned_text,
        'language': answer['language'],
        'main_character': answer['person']
    }
    dict_list.append(result) # Добавляем ответ в список
   # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

100%|██████████| 13/13 [00:16<00:00,  1.28s/it]


In [61]:
import pandas as pd

# Создаем DataFrame из результатов
results_df = pd.DataFrame(dict_list)

# Добавляем колонку с исходными текстами (если у вас есть список текстов)
# Предположим, что texts - это список исходных текстов в том же порядке
texts = df['raw_text'].tolist()[:len(results_df)]  # берем столько текстов, сколько результатов

# Создаем итоговый DataFrame
final_df = pd.DataFrame({
    'text': results_df['cleaned_text'],
    'language': results_df['language'],
    'main_character': results_df['main_character']
})


Сохраним всё в итоговый файл. Убедитесь, что на этом этапе у вас в столбцах

- `text` - очищенный текст (без символов ¿, ¡, £)
- `language` - язык, на котором написан текст (название языка указать на английском языке)
- `main_character` - имя главного персонажа в тексте (указать на том языке, на котором и написан сам текст)

In [62]:
final_df

,text,language,main_character
0,"The sun was setting, casting long shadows over...",English,John
1,"Le soleil se couchait, jetant de longues ombre...",French,Pierre
2,"El sol se estaba poniendo, proyectando largas ...",Spanish,Carlos
3,"La ciudad estaba llena de vida, sus calles lle...",Spanish,Juan
4,"La ville était pleine de vie, ses rues remplie...",French,Jean
5,"Die Stadt war voller Leben, ihre Straßen gefül...",German,Johann
6,Die Sonne ging unter und warf lange Schatten ü...,German,Hans
7,"В тихом уголке старого города, где узкие улочк...",Russian,Анна
8,In a small town nestled between the mountains ...,English,Laura
9,En un pequeño pueblo situado entre las montaña...,Spanish,Maria


In [64]:
final_df[['text', 'language', 'main_character']].to_csv('3.2.9_solution.csv', index=False)

In [ ]:
import sys
import traceback
import random
from IPython.core.magic import register_cell_magic
from IPython import get_ipython

# Сохраняем оригинальный механизм выполнения
original_run_cell = None

def setup_support_agent():
    """Устанавливает агента поддержки для всех ячеек"""

    # Поддерживающие сообщения
    supportive_messages = [
        "💪 Ты справишься! Ошибки - это часть пути к успеху!",
        "🌟 Даже профи когда-то делали такие ошибки!",
        "🚀 Ты уже почти решил задачу! Не сдавайся!",
        "🎯 Каждая ошибка делает тебя лучше! Продолжай!",
        "🦸‍♂️ Ты кодовый супергерой! У тебя всё получится!",
        "💫 Не переживай, это мелочь для тебя!",
        "🔥 Ты на правильном пути! Продолжай в том же духе!"
    ]

    def show_support(error_type, error_msg, tb):
        """Показывает поддержку при ошибке"""

        print("="*60)
        print(f"🚨 ОШИБКА")

        # Показываем поддержку
        print(f"   {random.choice(supportive_messages)}")
        print("="*60 + "\n")

    # Перехватываем все исключения в IPython
    def custom_exception_handler(shell, etype, value, tb, tb_offset=None):
        # Показываем обычный traceback
        shell.showtraceback((etype, value, tb), tb_offset=tb_offset)

        # Показываем поддержку
        show_support(etype.__name__, str(value), tb)


    # Устанавливаем обработчик
    ipython = get_ipython()
    if ipython:
        ipython.set_custom_exc((Exception,), custom_exception_handler)
        print("✅ АГЕНТ ПОДДЕРЖКИ АКТИВИРОВАН!")
        print("📝 Теперь ты можешь запускать код как обычно!")
        print("🤖 При ошибках я буду помогать и поддерживать тебя!")
    else:
        print("❌ Не удалось активировать агента")

# Активируем агента
setup_support_agent()

✅ АГЕНТ ПОДДЕРЖКИ АКТИВИРОВАН!
📝 Теперь ты можешь запускать код как обычно!
🤖 При ошибках я буду помогать и поддерживать тебя!


In [ ]:
print(undefined_variable)

NameError: name 'undefined_variable' is not defined

🚨 ОШИБКА
   🔥 Ты на правильном пути! Продолжай в том же духе!

